### In this notebook we explored covid papers dataset. 
For this particular notebook we have explored different alpha values for LDA Model. We are doing text analysis on paper Abstracts & Titles using nltk for text preprocessing, Topic modeling using LDA, Paper similarity analysis using TF-IDF vectors.

In [1]:
# Gensim
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pandas as pd
import numpy as np
np.random.seed(2020)

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

/opt/conda/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
import numpy as np
df = pd.read_excel('AbstractsCleanUpdated_v1.xlsx');
#data = pd.read_excel('AbstractsCleanUpdated_v1.xlsx', usecols=['Publication Date','Title','Abstract']);
df.head(5)

,ID,DOI,Latest Version,License Type,PMCID,PMID,Pub Year,Publication Date,Publication Types,Published As,...,Total Citations,Total References,ISSN,Journal Country,Journal Issue,Journal Name,Journal Name Full,Journal Pages,Journal Volume,System ID
0,15971,10.1177/2150132720936858,Yes,NaN,NaN,32578477.0,2020,2020-01-01,Journal Article,NaN,...,NaN,NaN,2150-1319,United States,NaN,J Prim Care Community Health,Journal of primary care & community health,2150132720936860,11,32578477
1,22120,10.1093/jtm/taaa008,Yes,NaN,PMC7107534,31943059.0,2020,2020-01-17,Journal Article,NaN,...,46.0,3.0,1195-1982,England,2,J Travel Med,Journal of travel medicine,NaN,27,31943059
2,18664,10.1101/2020.01.19.911669,Yes,It is made available under a CC-BY-NC-ND 4.0 I...,NaN,NaN,2020,2020-01-19,Preprint,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.1101/2020.01.19.911669
3,1781,10.2139/ssrn.3523861,Yes,NaN,NaN,NaN,2020,2020-01-21,Preprint,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2139/ssrn.3523861
4,13155,10.1101/2020.01.22.915660,No,NaN,PMC7217099,32511294.0,2020,2020-01-22,Preprint,Now published as: 32094589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.1101/2020.01.22.915660


In [4]:
df=df.dropna(axis=0, subset=['Publication Date','Title', 'Abstract',])

In [ ]:
data = df

In [5]:
data.tail()

,ID,DOI,Latest Version,License Type,PMCID,PMID,Pub Year,Publication Date,Publication Types,Published As,...,Total Citations,Total References,ISSN,Journal Country,Journal Issue,Journal Name,Journal Name Full,Journal Pages,Journal Volume,System ID
62449,136676,10.1016/j.medin.2020.09.002,Yes,NaN,NaN,33158594.0,2020,2020-11-08,Journal Article,NaN,...,NaN,NaN,0210-5691,Spain,NaN,Med Intensiva,Medicina intensiva,NaN,NaN,33158594
62450,136677,10.1007/s00277-020-04328-4,Yes,NaN,NaN,33159569.0,2020,2020-11-08,Journal Article,NaN,...,NaN,NaN,0939-5555,Germany,NaN,Ann Hematol,Annals of hematology,NaN,NaN,33159569
62451,136706,10.1097/MD.0000000000023044,Yes,NaN,NaN,33157957.0,2020,2020-11-08,Journal Article,NaN,...,NaN,NaN,0025-7974,United States,45,Medicine (Baltimore),Medicine,e23044,99,33157957
62452,136707,10.1186/s40792-020-01054-x,Yes,NaN,NaN,33159457.0,2020,2020-11-08,Journal Article,NaN,...,NaN,NaN,2198-7793,Germany,1,Surg Case Rep,Surgical case reports,281,6,33159457
62453,136708,10.1007/s11239-020-02335-w,Yes,NaN,NaN,33159639.0,2020,2020-11-08,Journal Article,NaN,...,NaN,NaN,0929-5305,Netherlands,NaN,J Thromb Thrombolysis,Journal of thrombosis and thrombolysis,NaN,NaN,33159639


## Test Pre-Processing
### Includes Stopword Removal(we also have extended list of stopwords), Tokenization, Building Biagrams and Triagrams, Lamatization etc.

In [6]:
with open('StopWordsList.txt', 'r') as s:
    sw = s.readlines()
    stopwords_data = list(map(lambda x: x.replace('\n',''), sw))
    stopwords_data[0] = '$ 0.002'
    
    
# NLTK Stop words
stop_words = stopwords.words('english')

#extending stopwords with some custom stopwords
stop_words.extend(stopwords_data)

In [7]:
data = data.Abstract.values.tolist()

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['objective', 'social', 'distancing', 'and', 'hand', 'washing', 'with', 'soap', 'and', 'water', 'have', 'been', 'advocated', 'as', 'the', 'main', 'proactive', 'measures', 'against', 'the', 'spread', 'of', 'coronavirus', 'we', 'sought', 'to', 'find', 'out', 'what', 'other', 'alternative', 'materials', 'and', 'methods', 'would', 'be', 'used', 'among', 'populations', 'without', 'running', 'water', 'and', 'who', 'may', 'not', 'afford', 'alcohol', 'based', 'sanitizers', 'results', 'we', 'reviewed', 'studies', 'that', 'reported', 'use', 'of', 'sand', 'soil', 'ash', 'soda', 'ash', 'seawater', 'alkaline', 'materials', 'and', 'sunlight', 'as', 'possible', 'alternatives', 'to', 'handwashing', 'with', 'soap', 'and', 'water', 'we', 'identified', 'the', 'documented', 'mechanism', 'of', 'actions', 'of', 'these', 'alternative', 'wash', 'methods', 'on', 'both', 'inanimate', 'surfaces', 'and', 'at', 'cellular', 'levels', 'the', 'consideration', 'of', 'use', 'of', 'these', 'alternative', 'locally', 'av

In [9]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# To get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Print trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['objective', 'social_distancing', 'and', 'hand_washing', 'with', 'soap', 'and', 'water', 'have', 'been', 'advocated', 'as', 'the', 'main', 'proactive', 'measures', 'against', 'the', 'spread', 'of', 'coronavirus', 'we', 'sought', 'to', 'find', 'out', 'what', 'other', 'alternative', 'materials', 'and', 'methods', 'would', 'be', 'used', 'among', 'populations', 'without', 'running_water', 'and', 'who', 'may', 'not', 'afford', 'alcohol', 'based', 'sanitizers', 'results', 'we', 'reviewed', 'studies', 'that', 'reported', 'use', 'of', 'sand', 'soil', 'ash', 'soda', 'ash', 'seawater', 'alkaline', 'materials', 'and', 'sunlight', 'as', 'possible', 'alternatives', 'to', 'handwashing', 'with', 'soap', 'and', 'water', 'we', 'identified', 'the', 'documented', 'mechanism', 'of', 'actions', 'of', 'these', 'alternative', 'wash', 'methods', 'on', 'both', 'inanimate_surfaces', 'and', 'at', 'cellular', 'levels', 'the', 'consideration', 'of', 'use', 'of', 'these', 'alternative', 'locally', 'available', 'in

In [10]:

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):

    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['objective', 'soap', 'water', 'advocate', 'main', 'proactive', 'measure', 'spread', 'seek', 'alternative', 'material', 'method', 'population', 'afford', 'alcohol', 'sanitizer', 'review', 'report', 'sand', 'soil', 'ash', 'ash', 'seawater', 'alkaline', 'material', 'sunlight', 'possible', 'alternative', 'handwash', 'soap', 'identify', 'document', 'mechanism', 'action', 'alternative', 'wash', 'method', 'cellular', 'level', 'consideration', 'alternative', 'locally', 'situation', 'unavailability', 'soap', 'water', 'alcohol', 'sanitizer', 'timely', 'face', 'pandemic', 'randomized', 'carry', 'evaluate', 'effectiveness', 'alternative', 'management']]


### Creating Dictionary ,Corpus and TF

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 5), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2)]]


In [13]:
# Term-frequency
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('action', 1),
  ('advocate', 1),
  ('afford', 1),
  ('alcohol', 2),
  ('alkaline', 1),
  ('alternative', 5),
  ('ash', 2),
  ('carry', 1),
  ('cellular', 1),
  ('consideration', 1),
  ('document', 1),
  ('effectiveness', 1),
  ('evaluate', 1),
  ('face', 1),
  ('handwash', 1),
  ('identify', 1),
  ('level', 1),
  ('locally', 1),
  ('main', 1),
  ('management', 1),
  ('material', 2),
  ('measure', 1),
  ('mechanism', 1),
  ('method', 2),
  ('objective', 1),
  ('pandemic', 1),
  ('population', 1),
  ('possible', 1),
  ('proactive', 1),
  ('randomized', 1),
  ('report', 1),
  ('review', 1),
  ('sand', 1),
  ('sanitizer', 2),
  ('seawater', 1),
  ('seek', 1),
  ('situation', 1),
  ('soap', 3),
  ('soil', 1),
  ('spread', 1),
  ('sunlight', 1),
  ('timely', 1),
  ('unavailability', 1),
  ('wash', 1),
  ('water', 2)]]

## Building LDA Mallet Model

In [14]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
    

--2020-12-21 18:02:58--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  8.10MB/s    in 1.9s    

2020-12-21 18:03:00 (8.10 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [15]:
!unzip ./mallet-2.0.8.zip

Archive:  ./mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classify  
  inflating: mallet-2.0.8/bin/text2vectors  
  inflating: mallet-2.0.8/bin/vectors2classify  
  inflating: mallet-2.0.8/bin/vectors2info  
  inflating: mallet-2.0.8/bin/vectors2topics  
  inflating: mallet-2.0.8/bin/vectors2vectors  
  inflating: mallet-2.0.8/build.xml  
   creating: mallet-2.0.8/class/
   creating: mallet-2.0.8/class/cc/
   creating: mallet-2.0.8/class/cc/mallet/
   creating: mallet-2.0.8/class/cc/mallet/classify/
  inflating: mallet-2.0.8/class/cc/mallet/classify/AdaBoost.class  
  

  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/GELattice.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/GELatticeTask.class  
   creating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/CachedDotTransitionIterator.class  
   creating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/OneLabelL2IndPRConstraints$OneLabelL2IndPRConstraint.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/OneLabelL2IndPRConstraints.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/OneLabelL2PRConstraints$OneLabelPRConstraint.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/OneLabelL2PRConstraints.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/semi_supervised/pr/constraints/PRConstraint.class  
  in

   creating: mallet-2.0.8/class/cc/mallet/pipe/tsf/
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/CountMatches.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/CountMatchesAlignedWithOffsets.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/CountMatchesMatching.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/FeaturesInWindow.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/FeaturesOfFirstMention.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/LexiconMembership.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/OffsetConjunctions.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/OffsetFeatureConjunction.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/OffsetPropertyConjunctions.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/RegexMatches.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/SequencePrintingPipe.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/tsf/Target2BIOFormat.cla

  inflating: mallet-2.0.8/dist/mallet-deps.jar  
  inflating: mallet-2.0.8/dist/mallet.jar  
   creating: mallet-2.0.8/lib/
  inflating: mallet-2.0.8/lib/bsh.jar  
  inflating: mallet-2.0.8/lib/derby.jar  
  inflating: mallet-2.0.8/lib/grmm-deps.jar  
  inflating: mallet-2.0.8/lib/jdom-1.0.jar  
  inflating: mallet-2.0.8/lib/jgrapht-0.6.0.jar  
  inflating: mallet-2.0.8/lib/junit-4.5.jar  
  inflating: mallet-2.0.8/lib/jwnl-1.3.jar  
  inflating: mallet-2.0.8/lib/LICENSES  
  inflating: mallet-2.0.8/lib/Makefile  
  inflating: mallet-2.0.8/lib/mallet-deps.jar  
  inflating: mallet-2.0.8/lib/mtj-0.9.9.jar  
  inflating: mallet-2.0.8/lib/openjgraph.jar  
  inflating: mallet-2.0.8/lib/trove-2.0.2.jar  
  inflating: mallet-2.0.8/LICENSE    
  inflating: mallet-2.0.8/Makefile   
  inflating: mallet-2.0.8/pom.xml    
  inflating: mallet-2.0.8/README.md  
   creating: mallet-2.0.8/sample-data/
   creating: mallet-2.0.8/sample-data/numeric/
  inflating: mallet-2.0.8/sample-data/numeric/boxes.t

  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/ConstrainedForwardBackwardConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/ConstrainedViterbiTransducerCorrector.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/GammaAverageConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/GammaProductConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/InstanceWithConfidence.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/IsolatedSegmentTransducerCorrector.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/MaxEntConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/MaxEntSequenceConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/MinSegmentConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confidence/NBestViterbiConfidenceEstimator.java  
  inflating: mallet-2.0.8/src/cc/mallet/fst/confiden

  inflating: mallet-2.0.8/src/cc/mallet/pipe/CharSequenceReplace.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/CharSubsequence.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/Classification2ConfidencePredictingFeatureVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/Csv2Array.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/Csv2FeatureVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/Directory2FileIterator.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureCountPipe.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureDocFreqPipe.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureSequence2AugmentableFeatureVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureSequence2FeatureVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureSequenceConvolution.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureValueString2FeatureVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/pipe/FeatureVectorConjunctions.ja

In [16]:
import os
os.environ.update({'MALLET_HOME': './mallet-2.0.8'})
mallet_path = './mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=11, id2word=id2word)

In [17]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(5, [('health', 0.031017108214853886), ('pandemic', 0.02290337798164217), ('care', 0.015537410732657169), ('public', 0.01224022764113399), ('community', 0.011388329878902545), ('system', 0.010309343022468019), ('service', 0.008973155810720612), ('challenge', 0.008460009735974425), ('response', 0.00762567687605703), ('crisis', 0.007372240428382875)]), (3, [('covid', 0.08051182901778707), ('infection', 0.07767983263280805), ('case', 0.06772300246807968), ('report', 0.042883531130401185), ('disease', 0.03997909634824242), ('symptom', 0.038444092217529056), ('respiratory', 0.03382183258796522), ('severe', 0.019813626352399003), ('child', 0.017364518638227123), ('virus', 0.015729480530456034)]), (8, [('patient', 0.07880031485680071), ('covid', 0.04997880771533716), ('risk', 0.029569628428608913), ('high', 0.028197175707586838), ('mortality', 0.02727009734995526), ('age', 0.0263605110368073), ('group', 0.023081425467071225), ('compare', 0.01826707660842713), ('outcome', 0.016726431151985684

In [19]:
from pprint import pprint 
# Select the model with 11 topics and print the topics with 50 words
mallet_model_topics = ldamallet.show_topics(formatted=False)
pprint(ldamallet.print_topics(num_words=50))

[(0,
  '0.061*"test" + 0.022*"sample" + 0.021*"testing" + 0.020*"negative" + '
  '0.018*"method" + 0.017*"time" + 0.016*"high" + 0.016*"covid" + '
  '0.014*"detect" + 0.013*"perform" + 0.012*"detection" + 0.012*"low" + '
  '0.012*"diagnostic" + 0.011*"diagnosis" + 0.010*"evaluate" + '
  '0.010*"sensitivity" + 0.009*"individual" + 0.009*"collect" + '
  '0.008*"laboratory" + 0.008*"antibody" + 0.008*"viral" + 0.008*"clinical" + '
  '0.007*"rapid" + 0.007*"compare" + 0.007*"performance" + 0.007*"reaction" + '
  '0.006*"screen" + 0.006*"assess" + 0.006*"image" + 0.006*"determine" + '
  '0.006*"specific" + 0.006*"screening" + 0.005*"early" + 0.005*"real" + '
  '0.005*"point" + 0.005*"pcr" + 0.005*"identify" + 0.005*"specificity" + '
  '0.005*"demonstrate" + 0.005*"load" + 0.005*"control" + 0.005*"total" + '
  '0.005*"conclusion" + 0.004*"accuracy" + 0.004*"prevalence" + 0.004*"set" + '
  '0.004*"subject" + 0.004*"range" + 0.004*"virus" + 0.004*"rate"'),
 (1,
  '0.039*"model" + 0.033*"case" 

### Most representative document for each topic

In [23]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.2242,"patient, care, hospital, pandemic, risk, medic...","[objective, soap, water, advocate, main, proac..."
1,1,3.0,0.1167,"covid, infection, case, report, disease, sympt...","[still, several, unanswered_question, infectio..."
2,2,1.0,0.1894,"model, case, country, rate, measure, spread, p...","[report, novel, identify, causative, virus, ch..."
3,3,2.0,0.2228,"patient, covid, acute, respiratory, severe, lu...","[novel, associate, clinical, method, case, adm..."
4,4,9.0,0.4136,"cell, virus, viral, human, protein, response, ...","[past, year, several, coronaviruse, cross, spe..."
5,5,9.0,0.3063,"cell, virus, viral, human, protein, response, ...","[detailed, genomic, structure, analysis, new, ..."
6,6,9.0,0.3094,"cell, virus, viral, human, protein, response, ...","[year, ago, large, severe, acute, respiratory,..."
7,7,9.0,0.3435,"cell, virus, viral, human, protein, response, ...","[cause, novel, designate, determine, sequence,..."
8,8,1.0,0.3357,"model, case, country, rate, measure, spread, p...","[case, infection, quickly, spread, province, n..."
9,9,1.0,0.2658,"model, case, country, rate, measure, spread, p...","[notify, city, authority, later, identify, new..."


In [52]:
def get_results():
    temp = []
    for d in df_dominant_topic.Document_No:
        doc_id = df.iloc[d]['ID']
        topic_id = 'Topic'+ str(int(df_dominant_topic.iloc[d]['Dominant_Topic'])+1)
        #print(doc_id, topic_id)
        temp.append([doc_id, topic_id])
    results = pd.DataFrame(temp)
    results.columns = ['ID', 'LDA_TOPIC']
    return results.sort_values(by=['ID'])

results = get_results()
results.head()

,ID,LDA_TOPIC
19652,1,Topic1
7133,2,Topic10
13633,3,Topic6
19448,4,Topic3
8433,5,Topic8


In [53]:
results.to_excel('LDA_Topic_with_Id.xlsx', index=False)